In [ ]:
import os, subprocess
from tools import dataset_tools
import pandas as pd
import numpy as np
import config, models
import multiprocessing

# Extract features from original graph

In [ ]:
# main variables
dataset_name               = "FB13"
embedding_model            = models.TransE
model_timestamp            = '1524490825'

# GPU settings
# os.environ["CUDA_VISIBLE_DEVICES"] = "1" # should be a string

In [ ]:
dataset_path = './benchmarks/{}/'.format(dataset_name)

## Create original graph for PRA

In [ ]:
from tools.pra_setup import create_graph_input

create_graph_input(
    dataset_path,
    labels=['valid.tsv', 'test.tsv'] # folds that have labels
)

The command below will automatically run `ExperimentRunner`, without asking for class selection, for the specific dataset.

Make sure the benchmark (original) graph is created.

**Warning**: the cell below will return an error, but the files needed probably will be generated correcly. This is happening because the `.json` spec files don't have all atributes, this is something to fix later.

In [ ]:
%%bash -s "$dataset_name"
(cd /home/arthurcgusmao/Projects/xkbc/algorithms/pra/; sbt "runMain edu.cmu.ml.rtw.pra.experiments.ExperimentRunner /home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/benchmarks/pra/ $1")

Generate split (inside `./results/`) with random negative examples (bernoulli or uniform)

## Generate/Read Negative Examples

In [ ]:
neg_rate = 2 # negative to positive ratio
bern = True

In [ ]:
distribution = 'bern' if bern else 'unif'
corrupted_filename = 'train2id_{}_{}to1.txt'.format(distribution, neg_rate)
corrupted_dirpath = dataset_path + '/corrupted/'
corrupted_filepath = corrupted_dirpath + corrupted_filename

In [ ]:
if not os.path.exists(corrupted_filepath):
    # create corrupted dirpath if not exist
    if not os.path.exists(corrupted_dirpath):
        os.makedirs(corrupted_dirpath)
    # generate corrupted set and save to disk in `corrupted` folder
    corrupted = dataset_tools.generate_corrupted_training_examples(dataset_path,
            neg_proportion=neg_rate, bern=bern)
    train2id = pd.DataFrame(corrupted)
    train2id.to_csv(corrupted_filepath,
        columns=['head', 'tail', 'relation', 'label'], index=False, header=False, sep=' ')
    print('Created corrupted file: {}.'.format(corrupted_filepath))    
else:
    train2id = pd.read_csv(corrupted_filepath,
        names=['head', 'tail', 'relation', 'label'], sep=' ', skiprows=0)
    print('Corrupted file already exists: {}.'.format(corrupted_filepath))

## Read validation and test examples

In [ ]:
if dataset_name == 'FB15k' :
    valid2id = pd.read_csv(dataset_path + 'valid_neg.txt', sep='\t', skiprows=0, names=['head', 'tail', 'relation', 'label'])
    test2id = pd.read_csv(dataset_path + 'test_neg.txt', sep='\t', skiprows=0, names=['head', 'tail', 'relation', 'label'])
else:
    valid2id_pos = pd.read_csv(dataset_path + 'valid2id.txt', sep=' ', skiprows=1, names=['head', 'tail', 'relation'])
    valid2id_neg = pd.read_csv(dataset_path + 'valid2id_neg.txt', sep=' ', skiprows=1, names=['head', 'tail', 'relation'])
    test2id_pos = pd.read_csv(dataset_path + 'test2id.txt', sep=' ', skiprows=1, names=['head', 'tail', 'relation'])
    test2id_neg = pd.read_csv(dataset_path + 'test2id_neg.txt', sep=' ', skiprows=1, names=['head', 'tail', 'relation'])

    valid2id_pos['label'] = 1
    valid2id_neg['label'] = -1
    test2id_pos['label'] = 1
    test2id_neg['label'] = -1

    valid2id = pd.concat((valid2id_pos, valid2id_neg))
    test2id = pd.concat((test2id_pos, test2id_neg))

In [ ]:
display(train2id.head())
display(valid2id.head())
display(test2id.head())

## Restore working model

In [ ]:
import_path = './results/{}/{}/{}/'.format(
    dataset_name,
    embedding_model.__name__,
    model_timestamp
)

model_info_df = pd.read_csv('{}/model_info.tsv'.format(import_path), sep='\t')
model_info_df.transpose()

In [ ]:
# transform model info into dict with only one "row"
model_info = model_info_df.to_dict()
for key,d in model_info.iteritems():
    model_info[key] = d[0]

In [ ]:
con = config.Config()
dataset_path = "./benchmarks/{}/".format(model_info['dataset_name'])
con.set_in_path(dataset_path)
con.set_test_link_prediction(False)
con.set_test_triple_classification(True)
con.set_work_threads(multiprocessing.cpu_count())
con.set_dimension(int(model_info['k']))
con.score_norm = model_info['score_norm']
con.init()
con.set_model(embedding_model)
con.import_variables("{}tf_model/model.vec.tf".format(import_path)) # loading model via tensor library

## Predict and Update Data

In [ ]:
for fold in [train2id, valid2id, test2id]:
    fold['label'] = con.classify(fold['head'], fold['tail'], fold['relation'])
    fold['label'] = fold['label'].map(lambda x: 1 if x==1 else -1)

## Decode from id to names

In [ ]:
entity2id, id2entity = dataset_tools.read_name2id_file(dataset_path + 'entity2id.txt')
relation2id, id2relation = dataset_tools.read_name2id_file(dataset_path + 'relation2id.txt')

for fold in [train2id, valid2id, test2id]:
    fold['head'] = fold['head'].map(id2entity)
    fold['tail'] = fold['tail'].map(id2entity)
    fold['relation'] = fold['relation'].map(id2relation)

# WARNING: at this stage we have transformed the dataframes,
#   and entities and relations are not represented by ids anymore

## Move PRA template to results (model) dir

In [ ]:
pra_explain_path = import_path + '/pra_explain/'
split_name = '{}_{}to1'.format(distribution, neg_rate)

Move template to `results`

In [ ]:
%%bash -s "$pra_explain_path"
cp ./tools/pra_explain_results_template/* $1/ -R

Change variables in `experiment_specs`

In [ ]:
%%bash -s "$dataset_name" "$pra_explain_path" "$split_name"
sed -i "s/_DATASETNAME_/$1/g" $2/experiment_specs/*
sed -i "s/_SPLITNAME_/$3/g" $2/experiment_specs/*

## Create Split

In [ ]:
from tools import pra_setup

pra_setup.create_split({'train': train2id, 'valid': valid2id, 'test': test2id},
                       splits_dirpath=import_path+'/pra_explain/splits',
                       split_name=split_name)

## Extract Paths for split

In [ ]:
%%bash -s "$pra_explain_path"
(cd /home/arthurcgusmao/Projects/xkbc/algorithms/pra/; sbt "runMain edu.cmu.ml.rtw.pra.experiments.ExperimentRunner /home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/$1")

In [ ]:
print("Features (paths) extracted and saved into:\n{}".format(os.path.abspath(pra_explain_path)))

# Debug

In [ ]:
# !rm /home/arthurcgusmao/Projects/xkbc/algorithms/OpenKE/./results/FB13/TransE/1524490825//pra_explain//results/ -r